# Tarea 2. Minería de Texto Básica

Procesamiento de Lenguaje Natural

Eric Lemus Avalos 


Cargamos los datos y funciones de la práctica 2, creamos un tokenizador con TweetTokenizer. Creamos un diccionario de frecuencias y un diccionario character-index. 


In [48]:
import os
import nltk 
import numpy as np  
from nltk.tokenize import TweetTokenizer
from practica2 import get_text_from_file, order_dic_frec

In [49]:
text_train , target_train = get_text_from_file("./data_mex/mex20_train.txt", "./data_mex/mex20_train_labels.txt")
text_val, target_val = get_text_from_file("./data_mex/mex20_val (1).txt", "./data_mex/mex20_val_labels (1).txt")

In [50]:
tk = TweetTokenizer()

corpus = []
for txt in text_train:
    corpus += tk.tokenize(txt)

In [94]:
fdist = nltk.FreqDist(corpus)
V = order_dic_frec(fdist)
V = V[:5000]

dict_index = dict()
count = 0
for weight, word in V:
    dict_index[word] = count
    count += 1


## Bolsas de palabras, Bigramas y Emociones

1. Evalué BoW con pesado binario.

In [52]:
def BoW(text, vocabulario, dict_index): 
    BOW = np.zeros((len(text),len(vocabulario)),dtype = int)
     
    cont_doc = 0 

    for doc in text: 
        fdist_doc = nltk.FreqDist(tk.tokenize(doc))

        #llenar la bolsa 
        for word in fdist_doc:
            if word in dict_index: 
                BOW[cont_doc,dict_index[word]] = 1 #frecuencia 1 para cuando el i-termino esta en el j-doc
        cont_doc += 1  
    
    return BOW 

In [53]:
BoW_bin_train = BoW(text_train,V,dict_index)
BoW_bin_val   = BoW(text_val,V,dict_index)

In [54]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score

parametros = {'C': [0.5, .12, .25, .5, 1, 2, 4]}

clasificador = svm.LinearSVC(class_weight = 'balanced')  
grid =  GridSearchCV(estimator = clasificador, param_grid = parametros, n_jobs = 8, scoring = "f1_macro",cv = 5)
grid.fit(BoW_bin_train, target_train)
y_pred = grid.predict(BoW_bin_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[358  60]
 [ 55 114]]
              precision    recall  f1-score   support

           0     0.8668    0.8565    0.8616       418
           1     0.6552    0.6746    0.6647       169

    accuracy                         0.8041       587
   macro avg     0.7610    0.7655    0.7632       587
weighted avg     0.8059    0.8041    0.8049       587



2. Evalué BoW con pesado frecuencia.

In [55]:
def BoW_frec(text, vocabulario, dict_index): 
    BOW = np.zeros((len(text),len(vocabulario)),dtype = int)
     
    cont_doc = 0 

    for doc in text: 
        fdist_doc = nltk.FreqDist(tk.tokenize(doc))

        #llenar la bolsa 
        for word in fdist_doc:
            if word in dict_index: 
                BOW[cont_doc,dict_index[word]] = fdist_doc[word]
        cont_doc += 1  
    
    return BOW 

In [95]:
BoW_frec_train = BoW_frec(text_train,V,dict_index)
BoW_frec_val   = BoW_frec(text_val,V,dict_index)

grid.fit(BoW_frec_train, target_train)
y_pred = grid.predict(BoW_frec_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[356  62]
 [ 54 115]]
              precision    recall  f1-score   support

           0     0.8683    0.8517    0.8599       418
           1     0.6497    0.6805    0.6647       169

    accuracy                         0.8024       587
   macro avg     0.7590    0.7661    0.7623       587
weighted avg     0.8054    0.8024    0.8037       587



3. Evalué BoW con pesado tfidf.

In [57]:
from collections import defaultdict

def BoW_tfidf(text, vocabulario, dict_index):
    BOW = np.zeros((len(text), len(vocabulario)), dtype=float)
    
    df = defaultdict(int)  # df[word] = num de doc donde aparece la palabra
    for doc in text:
        unique_words = set(tk.tokenize(doc))  
        for word in unique_words:
            if word in dict_index:
                df[word] += 1  
    
    cont_doc = 0
    T = len(text)  # total doc
    
    for doc in text:
        fdist_doc = nltk.FreqDist(tk.tokenize(doc))  
        
        for word in fdist_doc:
            if word in dict_index:
                tf = fdist_doc[word]
                idf = np.log(T / df[word])
                BOW[cont_doc, dict_index[word]] = tf * idf
        cont_doc += 1
    
    return BOW

In [58]:
BoW_TFidf_train = BoW_tfidf(text_train,V,dict_index)
BoW_TFidf_val   = BoW_tfidf(text_val,V,dict_index)

grid.fit(BoW_TFidf_train, target_train)
y_pred = grid.predict(BoW_TFidf_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[351  67]
 [ 78  91]]
              precision    recall  f1-score   support

           0     0.8182    0.8397    0.8288       418
           1     0.5759    0.5385    0.5566       169

    accuracy                         0.7530       587
   macro avg     0.6971    0.6891    0.6927       587
weighted avg     0.7484    0.7530    0.7504       587



4. Evalué BoW con pesado binario normalizado l2 (no use sklearn).

In [59]:
def norma_l2(bow):
    n,m = bow.shape 
    B_norm = np.zeros((n,m))
    
    for i in range(n):
        norm = np.linalg.norm(bow[i, :])
        if norm !=0:
            B_norm[i, :] = bow[i, :]/norm
    return B_norm

In [60]:
BoW_l2_train = norma_l2(BoW_bin_train)
BoW_l2_val   = norma_l2(BoW_bin_val)

grid.fit(BoW_l2_train, target_train)
y_pred = grid.predict(BoW_l2_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[356  62]
 [ 47 122]]
              precision    recall  f1-score   support

           0     0.8834    0.8517    0.8672       418
           1     0.6630    0.7219    0.6912       169

    accuracy                         0.8143       587
   macro avg     0.7732    0.7868    0.7792       587
weighted avg     0.8199    0.8143    0.8166       587



5. Evalué BoW con pesado frecuencia normalizado l2 (no use sklearn).

In [61]:
BoW_frecl2_train = norma_l2(BoW_frec_train)
BoW_frecl2_val   = norma_l2(BoW_frec_val)

grid.fit(BoW_frecl2_train, target_train)
y_pred = grid.predict(BoW_frecl2_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[358  60]
 [ 38 131]]
              precision    recall  f1-score   support

           0     0.9040    0.8565    0.8796       418
           1     0.6859    0.7751    0.7278       169

    accuracy                         0.8330       587
   macro avg     0.7950    0.8158    0.8037       587
weighted avg     0.8412    0.8330    0.8359       587



6. Evalué BoW con pesado tfidf normalizado l2 (no use sklearn).

In [62]:
BoW_TFidf_train = norma_l2(BoW_TFidf_train)
BoW_TFidf_val   = norma_l2(BoW_TFidf_val)

grid.fit(BoW_TFidf_train, target_train)
y_pred = grid.predict(BoW_TFidf_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[352  66]
 [ 39 130]]
              precision    recall  f1-score   support

           0     0.9003    0.8421    0.8702       418
           1     0.6633    0.7692    0.7123       169

    accuracy                         0.8211       587
   macro avg     0.7818    0.8057    0.7913       587
weighted avg     0.8320    0.8211    0.8248       587



7. Ponga una tabla comparativa a modo de resumen con las seis entradas anteriores.


| Método                           | Accuracy | Macro F1 | F1-score(0) | F1-score(1) |
|----------------------------------|----------|----------|------------|------------|
| **BoW Binario**                  | 0.8041   | 0.7632   | 0.8616     | 0.6647     |
| **BoW Frecuencia**                | 0.8024   | 0.7623   | 0.8599     | 0.6647     |
| **BoW TF-IDF**                    | 0.7530   | 0.6927   | 0.8288     | 0.5566     |
| **BoW Binario (L2 Norm.)**        | 0.8143   | 0.7792   | 0.8672     | 0.6912     |
| **BoW Frecuencia (L2 Norm.)**     | 0.8330   | 0.8037   | 0.8796     | 0.7278     |
| **BoW TF-IDF (L2 Norm.)**         | 0.8211   | 0.7913   | 0.8702     | 0.7123     |

8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos
(e.g., 1000 y 7000). Ponga una tabla dónde compare tres configuraciones distintas.

In [63]:
# Con 1000 terminos
V = order_dic_frec(fdist)
V_small = V[:1000]

dict_index = dict()
count = 0
for weight, word in V_small:
    dict_index[word] = count
    count += 1

# Elegimos BoW con pesado frecuencia norma l2
BoW_frec_train = BoW_frec(text_train,V_small,dict_index)
BoW_frec_val   = BoW_frec(text_val,V_small,dict_index)
BoW_frecl2_train = norma_l2(BoW_frec_train)
BoW_frecl2_val   = norma_l2(BoW_frec_val)

grid.fit(BoW_frecl2_train, target_train)
y_pred = grid.predict(BoW_frecl2_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[346  72]
 [ 39 130]]
              precision    recall  f1-score   support

           0     0.8987    0.8278    0.8618       418
           1     0.6436    0.7692    0.7008       169

    accuracy                         0.8109       587
   macro avg     0.7711    0.7985    0.7813       587
weighted avg     0.8252    0.8109    0.8154       587



In [64]:
# Con 7000 terminos
V = order_dic_frec(fdist)
V_big = V[:7000]

dict_index = dict()
count = 0
for weight, word in V_big:
    dict_index[word] = count
    count += 1

# Elegimos BoW con pesado frecuencia norma l2
BoW_frec_train = BoW_frec(text_train,V_big,dict_index)
BoW_frec_val   = BoW_frec(text_val,V_big,dict_index)
BoW_frecl2_train = norma_l2(BoW_frec_train)
BoW_frecl2_val   = norma_l2(BoW_frec_val)

grid.fit(BoW_frecl2_train, target_train)
y_pred = grid.predict(BoW_frecl2_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))


[[359  59]
 [ 39 130]]
              precision    recall  f1-score   support

           0     0.9020    0.8589    0.8799       418
           1     0.6878    0.7692    0.7263       169

    accuracy                         0.8330       587
   macro avg     0.7949    0.8140    0.8031       587
weighted avg     0.8403    0.8330    0.8357       587



**Comparación BoW pesado de frecuencia (normalizado  l2) con diferentes tamaños de vocabulario**




| Vocabulario | Accuracy | Macro F1 | F1-score(0) | F1-score(1) |
|------------|----------|----------|------------|------------|
| **1000**   | 0.8109   | 0.7813   | 0.8618     | 0.7008     |
| **5000**   | 0.8330   | 0.8037   | 0.8796     | 0.7278     |
| **7000**   | 0.8330   | 0.8031   | 0.8799     | 0.7263     |


9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado
"EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para
construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex en
Español). Para esto, una estrategia sencilla sería enmascarar (sustituir) cada palabra con
su emoción, y después construir la Bolsa de Emociones (BoE).

In [65]:
# cargamos data
path = r"C:\Users\ericl\Downloads\NRC-Emotion-Lexicon\NRC-Emotion-Lexicon\OneFilePerLanguage\Spanish-NRC-EmoLex.txt"

EmoLex = dict() #{palabra,puntaje emociones}

with open(path, "r", encoding="utf-8") as f:
    next(f)
    for line in f:
        cols = line.strip().split("\t")  # Dividir por tabulación y limpiar espacios
        spanish_word = cols[-1]  
        emotions = list(map(int, cols[1:11]))
        EmoLex[spanish_word] = emotions


In [66]:
# EmoLex

In [67]:
# list(EmoLex.items())[:5]

In [68]:
dict_emotions = {0:"enojo",1:"anticipación",2:"disgusto",3:"miedo",4:"alegría",
                 5:"negativo",6:"positivo",7:"tristeza",8:"sorpresa",9:"confianza"}

In [99]:
import spacy
from nltk.text import Text
from unidecode import unidecode
import nltk

nlp = spacy.load('es_core_news_sm')

def mask(text_train, dict_emotions, lexico):
    emot_mask = []  
    
    for txt in text_train:
        doc = nlp(unidecode(txt.lower()))
        tokens = [token.lemma_ for token in doc]
        
        emotions = []
        for word in tokens:
            if word in lexico:
                emotion_indx = [i for i, value in enumerate(lexico[word]) if value == 1]
                word_emotions = [dict_emotions[i] for i in emotion_indx]
                emotions.extend(word_emotions)
            else:
                emotions.append(word)
                
        emot_mask.append(Text(emotions))  
        
    return emot_mask

In [98]:
def get_emotion_for_word(word, lexico, dict_emotions):
    try:
        emotions = lexico[word]
        emotion_indices = [i for i, value in enumerate(emotions) if value == 1]
        
        if emotion_indices:
            return [dict_emotions[i] for i in emotion_indices]
        else:
            return ["sin emoción"]  
    except KeyError:
        return ["palabra no encontrada"]  

In [71]:
 # text_train[:10]
reversed_dict = {emotion: emotion_id for emotion_id, emotion in dict_emotions.items()}

In [72]:
emot_mask_train = mask(text_train,dict_emotions,EmoLex)
emot_mask_val = mask(text_val,dict_emotions,EmoLex)

In [73]:
# emot_mask_train[:10]

In [74]:
get_emotion_for_word('obeso',EmoLex,dict_emotions)

['disgusto', 'negativo']

In [75]:
def BoE(text, dict_emotions): 
    BOE = np.zeros((len(text),len(dict_emotions)),dtype = int)
     
    cont_doc = 0 
    for doc in text: 
        fdist_doc = nltk.FreqDist(doc)
        for word in fdist_doc:
            if word in dict_emotions: 
                BOE[cont_doc,dict_emotions[word]] = 1
        cont_doc += 1  
    
    return BOE 

In [76]:
# normalizamos ya que previamnete obtuvimos mejores resultandos
BOE_bin_train = norma_l2(BoE(emot_mask_train,reversed_dict))
BOE_bin_val = norma_l2(BoE(emot_mask_val,reversed_dict))

10. Usa tú BoE de alguna forma y clasifica con SVM. Ponga una tabla comparativa a modo
de resumen con tres pesados (bin, tf, tfidf), normalize cada uno si lo cree conveniente.

In [77]:
grid.fit(BOE_bin_train, target_train)
y_pred = grid.predict(BOE_bin_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[297 121]
 [ 95  74]]
              precision    recall  f1-score   support

           0     0.7577    0.7105    0.7333       418
           1     0.3795    0.4379    0.4066       169

    accuracy                         0.6320       587
   macro avg     0.5686    0.5742    0.5700       587
weighted avg     0.6488    0.6320    0.6393       587



In [78]:
def BoE_frec(text, dict_emotions): 
    BOE = np.zeros((len(text),len(dict_emotions)),dtype = int)
     
    cont_doc = 0 
    for doc in text: 
        fdist_doc = nltk.FreqDist(doc)
        for word in fdist_doc:
            if word in dict_emotions: 
                BOE[cont_doc,dict_emotions[word]] = fdist_doc[word]
        cont_doc += 1  
    
    return BOE 

In [79]:
BOE_frec_train = norma_l2(BoE_frec(emot_mask_train,reversed_dict))
BOE_frec_val = norma_l2(BoE_frec(emot_mask_val,reversed_dict))

grid.fit(BOE_frec_train, target_train)
y_pred = grid.predict(BOE_frec_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[294 124]
 [ 97  72]]
              precision    recall  f1-score   support

           0     0.7519    0.7033    0.7268       418
           1     0.3673    0.4260    0.3945       169

    accuracy                         0.6235       587
   macro avg     0.5596    0.5647    0.5607       587
weighted avg     0.6412    0.6235    0.6312       587



In [80]:
def BoE_tfidf(text, dict_emotions):
    BOE = np.zeros((len(text), len(dict_emotions)), dtype=float)
         
    df = {word: sum(1 for doc in text if word in doc) for word in dict_emotions}
            
    cont_doc = 0
    T = len(text)  
    for doc in text:
        fdist_doc = nltk.FreqDist(doc)  
        
        for word in fdist_doc:
            if word in dict_emotions:
                tf = fdist_doc[word]
                idf = np.log(T / df[word])
                BOE[cont_doc, dict_emotions[word]] = tf * idf
        cont_doc += 1
    
    return BOE

In [81]:
BOE_tfidf_train = norma_l2(BoE_tfidf(emot_mask_train,reversed_dict))
BOE_tfidf_val = norma_l2(BoE_tfidf(emot_mask_val,reversed_dict))

grid.fit(BOE_tfidf_train, target_train)
y_pred = grid.predict(BOE_tfidf_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[291 127]
 [ 97  72]]
              precision    recall  f1-score   support

           0     0.7500    0.6962    0.7221       418
           1     0.3618    0.4260    0.3913       169

    accuracy                         0.6184       587
   macro avg     0.5559    0.5611    0.5567       587
weighted avg     0.6382    0.6184    0.6269       587



Comparación de la Bolsa de Emociones con diferentes esquemas de pesado, para cada caso se normalizo utilizando la norma l2. 

| Pesado              | Accuracy | Macro F1 | F1-score(0) | F1-score(1) |
|--------------------|----------|----------|------------|------------|
| **Binario**  | 0.6320   | 0.5700   | 0.7333     | 0.4066    |
| **Frecuencia**  | 0.6235   | 0.5607   | 0.7268     | 0.3945     |
| **TF-IDF**   | 0.6184   | 0.5567   | 0.7221     | 0.3913    |

## Recurso Línguistico de Emociones Mexicano

1. Utilice el recurso léxico llamado "Spanish Emotion Lexicon (SEL)" del Dr. Grigori Sidorov,
profesor del Centro de Investigación en Computación (CIC) del Instituto Politécnico Nacional, para enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf). Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla comparativa a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente.

In [82]:
path = r"C:\Users\ericl\Downloads\SEL\SEL.txt"

SEL = {} #{palabra,(PFA, categoria)}

with open(path, "r", encoding="cp1252") as f:
    next(f)
    for line in f:
        parts = line.strip().split("\t")  
        if len(parts) == 3:
            word, PFA, cat = parts
            SEL.setdefault(word, []).append((float(PFA), cat))

In [83]:
def mask_SEL(text_train, lexico):
    emot_mask = []  
    for txt in text_train:
        doc = nlp(unidecode(txt.lower()))
        tokens = [token.lemma_ for token in doc]  
        emotions = []
        for word in tokens:
            if word in lexico:
                fil_emotions = [emotion for pfa, emotion in lexico[word] if pfa > 0.4]
                if fil_emotions:
                    emotion = max(lexico[word], key=lambda x: x[0])[1]
                    emotions.append(emotion)
            else:
                emotions.append(word)  
                
        emot_mask.append(Text(emotions))  
        
    return emot_mask

In [84]:
emotSEL_mask_train = mask_SEL(text_train,SEL)
emotSEL_mask_val = mask_SEL(text_val,SEL)

In [85]:
dict_emotions = {"Alegría" : 0,"Enojo" : 1,"Miedo" : 2,"Repulsión" : 3,"Sorpresa" : 4,"Tristeza" : 5}

In [86]:
BOE_SEL_bin_train = norma_l2(BoE(emotSEL_mask_train,dict_emotions))
BOE_SEL_bin_val = norma_l2(BoE(emotSEL_mask_val,dict_emotions))

grid.fit(BOE_SEL_bin_train, target_train)
y_pred = grid.predict(BOE_SEL_bin_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[111 307]
 [ 39 130]]
              precision    recall  f1-score   support

           0     0.7400    0.2656    0.3908       418
           1     0.2975    0.7692    0.4290       169

    accuracy                         0.4106       587
   macro avg     0.5187    0.5174    0.4099       587
weighted avg     0.6126    0.4106    0.4018       587



In [87]:
BOE_SEL_frec_train = norma_l2(BoE_frec(emotSEL_mask_train,dict_emotions))
BOE_SEL_frec_val = norma_l2(BoE_frec(emotSEL_mask_val,dict_emotions))

grid.fit(BOE_SEL_frec_train, target_train)
y_pred = grid.predict(BOE_SEL_frec_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[111 307]
 [ 39 130]]
              precision    recall  f1-score   support

           0     0.7400    0.2656    0.3908       418
           1     0.2975    0.7692    0.4290       169

    accuracy                         0.4106       587
   macro avg     0.5187    0.5174    0.4099       587
weighted avg     0.6126    0.4106    0.4018       587



In [88]:
BOE_SEL_tfidf_train = norma_l2(BoE_tfidf(emotSEL_mask_train,dict_emotions))
BOE_SEL_tfidf_val = norma_l2(BoE_tfidf(emotSEL_mask_val,dict_emotions))

grid.fit(BOE_SEL_tfidf_train, target_train)
y_pred = grid.predict(BOE_SEL_tfidf_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[111 307]
 [ 40 129]]
              precision    recall  f1-score   support

           0     0.7351    0.2656    0.3902       418
           1     0.2959    0.7633    0.4264       169

    accuracy                         0.4089       587
   macro avg     0.5155    0.5144    0.4083       587
weighted avg     0.6086    0.4089    0.4006       587



Para cada caso se utilizo normalizado ya que mejoró los resultados. 

| Pesado          | Accuracy | Macro F1 | F1-score(0) | F1-score(1) |
|-----------------|----------|----------|-------------|-------------|
| **Binario**     | 0.4106   | 0.4099   | 0.3908      | 0.4290      |
| **Frecuencia**  | 0.4106   | 0.4099   | 0.3908      | 0.4290      |
| **TF-IDF**      | 0.4089    | 0.4083   | 0.3902      | 0.4264      |


2. Discuta sobre la estrategía que utilizó para incorporar el "Probability Factor of Affective use".

Se utilizó el __Probability Factor of Affective (PFA)__ para seleccionar las emociones más relevantes asociadas de cada palabra en los tweets, filtrando aquellas con un PFA superior a 0.4. Este umbral se eligió porque proporciona mejores resultados en precisión. Luego, se asignó la emoción con el PFA más alto a cada palabra, asegurando que solo se seleccionen emociones significativas. Aunque se podría considerar agregar más de una emoción cuando ambas tienen un PFA alto, no se observó una mejora significativa al hacerlo.



## ¿Podemos mejorar con Bigramas?

1. Hacer un experimento dónde concatene una buena BoW según sus experimentos anteriores con otra BoW construida a partir de los 1000 bigramas más frecuentes.

In [89]:
bigramas = nltk.bigrams(corpus)
bigramDist = nltk.FreqDist(bigramas)
V_bigram = order_dic_frec(bigramDist)
V_bigram = V_bigram[:1000]

dictBigram_index = dict()
count = 0
for weight, word in V_bigram:
    dictBigram_index[word] = count
    count += 1


In [90]:
def BoW_bigram(text, vocabulario, dict_index): 
    BOW = np.zeros((len(text),len(vocabulario)),dtype = int)
     
    cont_doc = 0 

    for doc in text: 
        fdist_doc = nltk.FreqDist(nltk.bigrams(doc))

        #llenar la bolsa 
        for word in fdist_doc:
            if word in dict_index: 
                BOW[cont_doc,dict_index[word]] = fdist_doc[word]
        cont_doc += 1  
    
    return BOW 

In [91]:
BoW_bigrams_train = BoW_bigram(text_train,V_bigram,dictBigram_index)
BoW_bigrams_val = BoW_bigram(text_val,V_bigram,dictBigram_index)

BoW_concatenada_train = norma_l2(np.concatenate((BoW_frec_train,BoW_bigrams_train), axis = 1))
BoW_concatenada_val = norma_l2(np.concatenate((BoW_frec_val,BoW_bigrams_val), axis = 1))

grid.fit(BoW_concatenada_train, target_train)
y_pred = grid.predict(BoW_concatenada_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[358  60]
 [ 38 131]]
              precision    recall  f1-score   support

           0     0.9040    0.8565    0.8796       418
           1     0.6859    0.7751    0.7278       169

    accuracy                         0.8330       587
   macro avg     0.7950    0.8158    0.8037       587
weighted avg     0.8412    0.8330    0.8359       587



No mejora, pero si consideramos los 5000 bigramas más fecuentes aumneta a 0.8339 en accurracy. 

2. Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bigramas; usted elige las dimensionalidades. Para construir la representación final del documento utilice la concatenación de las representaciones según sus observaciones (e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa de Sentimientos de Grigori), y aliméntelas a un SVM.

Consideramos la concatenación de las siguintes bolsas 

-Bolsa de palabras de dimensión 5000 con pesado de frecuencias.<br>
-Bolsa de bigramas de dimensión 1000 con pesado de frecuencias<br>
-Bolsa de emociones EmoLex con pesado de frecuencias<br>
-Bolsa de emociones SEL con pesado TF-IDF<br>


Al final se normalizaron. 

In [96]:
BOW_huge_train = norma_l2(np.concatenate((BoW_frec_train,BoW_bigrams_train,BOE_tfidf_train, BOE_SEL_frec_train),axis = 1))
BOW_huge_val = norma_l2(np.concatenate((BoW_frec_val,BoW_bigrams_val,BOE_tfidf_val,BOE_SEL_frec_val),axis = 1))

In [97]:
grid.fit(BOW_huge_train, target_train)
y_pred = grid.predict(BOW_huge_val)

p, r, f, _= precision_recall_fscore_support(target_val, y_pred, average='macro', pos_label=1)

print(confusion_matrix(target_val,y_pred))
print(metrics.classification_report(target_val,y_pred, digits=4))

[[355  63]
 [ 47 122]]
              precision    recall  f1-score   support

           0     0.8831    0.8493    0.8659       418
           1     0.6595    0.7219    0.6893       169

    accuracy                         0.8126       587
   macro avg     0.7713    0.7856    0.7776       587
weighted avg     0.8187    0.8126    0.8150       587



3. Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea breve: todo en NO más de dos párrafos.

El modelo BoW con esquema de pesado de frecuencias fue el que tuvo un mejor desempeño, el cual mejoró al normalizar por columnas la matriz de término-vocabulario. El agregar bigramas no mejoró mucho el rendimiento, aunque sí hubo una mejora mínima al considerar grandes cantidades de bigramas (alrededor de 10,000).
El uso de estrategias de pesado más ingeniosas no mostró resultados de mejora, como en el caso de la TF-IDF. Por otro lado, para las emociones, dado que eran pocas, obtuvimos representaciones más densas de los términos, aunque los resultados fueron inferiores a los obtenidos con estrategias más sencillas como la bolsa de palabras con pesado binario. Aumentar el tamaño del vocabulario mejoró los resultados, pero llegó a un punto donde no se encontraron mejoras significativas. Por lo tanto, no vale la pena considerar toda la información. 




